# Lab 5 - Bias and mitigation

In this lab we will explore the concepts of bias and fairness in developing models by revisiting the Titanic classification example you did in Lab 2. In Lab 2 we stepped through training a decision tree classifier, so that given a new hypothetical passenger we can make a prediction if they would be survive (or not) if they had been on the Titanic at the time the ship sank. Here, we will use a different type of classifier and identify potential biases with a simple metric that you have already come across before (accuracy) and try out some simple mitigation techniques.

To complete this lab:
1. Follow the instructions running the code when asked.
2. Discuss each question with a study partner.
3. You should keep notes for your answers in a separate document (you can use [this template for Lab 5](Lab5_answers_template.docx)).
4. When completed, show your answers to a teacher so they can provide feedback.

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn import tree
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Let's look again at the Titanic problem example, but this time we will use a SVM classifier.

In [ ]:
titantic_passengers = pd.read_csv("titanic.csv")
titantic_passengers

**Q1.** Simply by inspecting the features (columns) and looking at some of the values, what _potential_ biases can you identify? Think about the representation of different groups within the overall passenger list. 

Hopefully you identified that there may be several subgroups within the passenger list that might be over- or under-represented. Let's take a closer look by inspecting the distributions of values for various features.

In [ ]:
survived_by_gender = titantic_passengers.groupby('sex').survived.mean()
_ = survived_by_gender.plot.bar()
survived_by_gender

**Q2.** Looking at the different distributions, do these exhibit bias? If so, is it necessarily a bad thing?

Hopefully you can see that when you inspect the distributions, rarely would you find the value counts balanced - and perhaps this is not a bad thing. For example, let's say our Titanic data only told us something about the sex of the passengers and in their survival rates showed that 50% of all females and 50% of all males survived, and there were exactly the same number of total men and women on the ship. If that were the case, could we predict anything about survival based on this data? Probably not, since the probabilities are evenly split between sexes, 50/50, and the survival rates are perfectly balanced. A model trained on this data would only be able to pick randomly.

However, something that was discussed in the lecture on testing ML was how that sometimes certain groups represented within a dataset may not achieve the same predictive performance as others. The degree of equalness between groups when evaluating the performance of classifiers is called _accuracy parity_.

Accuracy parity measures whether the accuracy of the model is consistent across different groups. Disparities in accuracy can signal bias. This is what we will explore further in this lab.

Let's train our SVM classifier and inspect the overall accuracy of our survival prediction model.

In [ ]:
titantic_passengers['age_range'] = pd.cut(titantic_passengers.age, [0, 15, 80], labels=['child', 'adult'])
titantic_passengers.age = titantic_passengers.age.fillna(titantic_passengers.age.median())
titantic_passengers.fare = titantic_passengers.fare.fillna(titantic_passengers.fare.median())
titantic_passengers = pd.get_dummies(titantic_passengers, columns=['sex'], drop_first=True)
survived_data = titantic_passengers.survived  # save this for training later
titantic_passengers = titantic_passengers[['sex_male', 'fare', 'age', 'sibsp']]
X_train, X_test, y_train, y_test = train_test_split(titantic_passengers, survived_data, test_size=0.25, random_state=42)
print("Our training data has {} rows".format(len(X_train)))
print("Our test data has {} rows".format(len(X_test)))
classifier = SVC(random_state=88)
classifier.fit(X_train.values, y_train.values)

We can get a list of predicted outcomes (survived or not) based on our held-out test data.

In [ ]:
y_pred =  classifier.predict(X_test)
y_pred

Then, we can compare the predictions to the actual survival in our held-out test data to count the number of correct predictions that our classifier made. Recall from earlier in the course that the accuracy of a classifier is defined as the proportion of correct predictions made by the model. It can be calculated using the formula:

$$
\text{Accuracy} = \frac{\text{Number of Correct Predictions}}{\text{Total Number of Predictions}}
$$

In [ ]:
correct_predictions = y_pred == y_test
num_correct_predictions = correct_predictions.sum()  # sums all the True values in the correct_predictions series
num_correct_predictions

In [ ]:
total_predictions = len( ... )
total_predictions

In [ ]:
... / ...

**Q3:** What is the accuracy of the classifier? Provide your answer as a percentage to two decimal places. (Hint: If you haven't already, you can find the answer by filling in the ellipses above).

**Q4:** Do you think this accuracy will be the same for different subgroups of kinds of passengers?

Whatever conclusion you came to in Q4, let's now find out!

In [ ]:
# Select the male subset from our train and test data
X_test_male = X_test[X_test['sex_male'] == 1]
y_test_male = y_test[X_test['sex_male'] == 1]

y_pred_male = classifier.predict(X_test_male)
correct_predictions_male = y_pred_male == y_test_male
correct_predictions_male.sum()

In [ ]:
X_test_female = X_test[X_test['sex_male'] == 0]
y_test_female = y_test[X_test['sex_male'] == 0]

y_pred_female = classifier.predict(X_test_female)
correct_predictions_female = y_pred_female == y_test_female
correct_predictions_female.sum()

**Q5:** What is the accuracy for the male group?

**Q6:** What is the accuracy for the female group?

**Q7:** Has accuracy parity been achieved? (i.e. are the accuracies similar)

One strategy to try mitigate bias within groups is stratification. Stratification, in simple terms, refers to the process of dividing a dataset into different "strata" or subgroups, making sure that each subgroup is represented in proportion to its prevalence in the whole dataset. This technique is often used in sampling or when splitting data into training and testing sets for model development.

Imagine you have a bowl of mixed nuts containing almonds, cashews, and peanuts in different proportions. If you want to create a smaller sample that represents the whole bowl accurately, you would use stratification. You would take a handful that contains almonds, cashews, and peanuts in the same ratio as they are present in the whole bowl, not just a random handful that might contain only almonds or peanuts.

In the context of mitigating bias in machine learning and data analysis:

- Representative Sampling: Stratification ensures that each subgroup (e.g., different genders, age groups, income levels) in your dataset is proportionally represented. This is crucial in studies and models where it's important to capture the characteristics of the entire population without overemphasizing or underrepresenting any segment.

- Reducing Sampling Bias: Without stratification, there's a risk that your sample or training/test split might not accurately represent the broader population, leading to biased results. For instance, in a medical study, if you randomly select participants without considering age or gender, you might end up with a sample skewed towards a specific age group or gender, which could bias your study results.

Luckily for us, scikit-learn provides an option for when we do our test-train split to stratify by a particular feature. In this example, let's try straify on survival outcome, 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(titantic_passengers, survived_data, test_size=0.25, random_state=42, 
                                                    stratify=survived_data)
print("Our training data has {} rows".format(len(X_train)))
print("Our test data has {} rows".format(len(X_test)))

# Calculate the weights for each group
group_weights = X_train['sex_male'].value_counts(normalize=True).to_dict()
weights = X_train['sex_male'].apply(lambda x: 1 / group_weights[x])

classifier = SVC(random_state=88)
classifier.fit(X_train.values, y_train.values)

y_pred = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

**Q8:** What is the accuracy of the classifier using stratification on survival? Provide your answer as a percentage to two decimal places.

**Q9:** Did the classifier do better or worse when using stratification?

In [ ]:
# Select the male subset from our train and test data
X_test_male = X_test[X_test['sex_male'] == 1]
y_test_male = y_test[X_test['sex_male'] == 1]

y_pred_male = classifier.predict(X_test_male)
correct_predictions_male = y_pred_male == y_test_male
correct_predictions_male

In [ ]:
X_test_female = X_test[X_test['sex_male'] == 0]
y_test_female = y_test[X_test['sex_male'] == 0]

y_pred_female = classifier.predict(X_test_female)
correct_predictions_female = y_pred_female == y_test_female
correct_predictions_female

**Q10:** What is the accuracy for the male group when using the stratified classifer?

**Q11:** What is the accuracy for the female group when using the stratified classifer?

**Q12:** Has accuracy parity been achieved? If not, calculate whether using stratification has improved the accuracy parity. (Hint: Compare the differences between the accuracies, before and after stratification)

You may or may not have found that stratification was effective in this case (note: not all mitigation approaches necessarily work in all cases!). Another popular and simple bias mitigation method when it comes to addressing accuracy parity is _reweighing_.

Reweighing, in simple terms, is a technique used in data analysis and machine learning to adjust the importance (or weight) given to different samples (or data points) in a dataset. The main goal is to reduce or mitigate bias, especially in situations where certain groups are underrepresented or overrepresented.

To understand reweighing, imagine you're making a fruit salad with apples, oranges, and bananas, but you have a lot more apples than oranges and bananas. To balance the flavors, you might decide to give more importance to each orange and banana you add (since there are fewer of them) and less to each apple. In data analysis, this is like giving more "weight" to underrepresented groups in your dataset, so their influence on the analysis or model is proportional to the overrepresented groups.

In the context of mitigating bias, reweighing is useful for:

- Balancing Representation: It helps in balancing the representation of different groups in your data. For example, if a dataset has more men than women, reweighing can increase the influence of the women's data to balance it out.

- Fairer Outcomes: By ensuring that all groups are equally represented, reweighing can lead to fairer outcomes in machine learning models. This is particularly important in applications like hiring, loan approval, and healthcare, where biased data can lead to unfair or discriminatory outcomes.

Let's try out reweighing on our Titanic dataset and our SVM classifier. Here, we do two steps:

1. Counting Men and Women: First, we look at the data and calculate what percentage of the data is men and what percentage is women. This is like counting how many men and women are in a room to see which group is more common.

2. Assigning Weights: Next, we give each person in the dataset a 'weight' based on these percentages. If there are fewer women, each woman gets a higher weight, and if there are fewer men, each man gets a higher weight. Think of it like giving a microphone to people in a group discussion. If there are fewer women speaking, you give them a microphone so their voices are heard as loudly as the men's.

For example, consider the following (this data is _not_ taken from the actual Titanic dataset):

| Person | sex_male |
|--------|----------|
| A      | 1        |
| B      | 0        |
| C      | 1        |
| D      | 1        |
| E      | 0        |

In this dataset, we have 3 males and 2 females.

First, we calculate the proportion of each gender in the dataset.

- Males (1): 3 out of 5, which is $ \frac{3}{5} = 0.6 $
- Females (0): 2 out of 5, which is $ \frac{2}{5} = 0.4 $

Next, we assign weights to each individual based on these proportions. The weight is the inverse of the proportion for each gender.

- Weight for Males: $ \frac{1}{0.6} \approx 1.67 $
- Weight for Females: $ \frac{1}{0.4} = 2.5 $

Now, each individual in the dataset is assigned a weight based on their gender.

| Person | sex_male | Weight  |
|--------|----------|---------|
| A (M)  | 1        | 1.67    |
| B (F)  | 0        | 2.5     |
| C (M)  | 1        | 1.67    |
| D (M)  | 1        | 1.67    |
| E (F)  | 0        | 2.5     |

Here, males are given a lower weight because they are more common in the dataset, while females, being less common, receive a higher weight. This balancing act ensures that when analysing the data, both genders have an equal influence overall, despite their differing numbers in the dataset.

But you don't need to calculate the weights manually, as we can use Python to do it.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(titantic_passengers, survived_data, test_size=0.25, random_state=42)
print("Our training data has {} rows".format(len(X_train)))
print("Our test data has {} rows".format(len(X_test)))

# Calculate the weights for each group
group_weights = X_train['sex_male'].value_counts(normalize=True).to_dict()
weights = X_train['sex_male'].apply(lambda x: 1 / group_weights[x])

classifier = SVC(random_state=88)
classifier.fit(X_train.values, y_train.values, sample_weight=weights)

# Predict and evaluate (assuming you have X_test and y_test)
y_pred = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

**Q13:** What is the accuracy of the classifier using reweighing on sex? Provide your answer as a percentage to two decimal places.

**Q14:** Did the classifier do better or worse when using reweighing?

Now let's check the accuracy parity again, this time on the classifier trained on the reweighed data.

In [ ]:
...

**Q15:** What is the accuracy for the male group when using the reweighed classifer?

**Q16:** What is the accuracy for the female group when using the reweighed classifer?

**Q17:** Has accuracy parity been achieved? If not, calculate whether using reweighing has improved the accuracy parity, as compared to the stratification approach and the original model (with no mitigation strategy applied).

In this lab, we show how you can inspect one aspect of bias with a specific metric (accuracy parity) and some very simple approaches to attempting to mitigate any apparent discrepencies that could signal bias. However, there are much more advanced algorithms and tools, such as IBM's AIF360 (https://aif360.res.ibm.com/) and the Fairlearn (https://fairlearn.org/) project that was originally developed by Microsoft, to do this for us. 

**Extra task:** If you're feeling adventurous, try and go back to the start of the notebook and work through it again but choosing a different subset that you might think expresses potential biases.

---
**When you are finished, let a teacher know you are finished and to check if you have any questions about the lab.**

If you wish to save your work in this notebook, choose **Save and Checkpoint** from the **File** menu, then choose **Download as Notebook**  from the **File** menu and save it to your computer or USB stick.